### Step 1: Load the overlay

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")
from pynq.lib import MicroblazeLibrary

lib = MicroblazeLibrary(base.RPI, ['uart'])
device = lib.uart_open(14,15)

### Step 2: Initialize Webcam and HDMI Out

In [2]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [3]:
# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [ ]:
# initialize camera from OpenCV
import cv2

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


### Step 3: Show input frame on HDMI output

In [ ]:
#import cv2
import numpy as np
import math
import time

cmds=[]
flag=0
state=4
act = {1: [0x55,0x55,0x05,0x06,0x09,0x01,0x00],
       2: [0x55,0x55,0x05,0x06,0x0A,0x01,0x00],
       3: [0x55,0x55,0x05,0x06,0x0B,0x01,0x00],
       4: [0x55,0x55,0x05,0x06,0x0C,0x01,0x00],
       5: [0x55,0x55,0x05,0x06,0x0D,0x01,0x00],
       6: [0x55,0x55,0x05,0x06,0x0E,0x01,0x00],
       7: [0x55,0x55,0x05,0x06,0x0F,0x01,0x00],
       8: [0x55,0x55,0x05,0x06,0x10,0x01,0x00],
       9: [0x55,0x55,0x05,0x06,0x11,0x01,0x00],
       10:[0x55,0x55,0x05,0x06,0x12,0x01,0x00],
       11:[0x55,0x55,0x05,0x06,0x13,0x01,0x00],
       12:[0x55,0x55,0x05,0x06,0x14,0x01,0x00],
       13:[0x55,0x55,0x05,0x06,0x15,0x01,0x00],
       14:[0x55,0x55,0x05,0x06,0x16,0x01,0x00],
       15:[0x55,0x55,0x05,0x06,0x17,0x01,0x00],
       16:[0x55,0x55,0x05,0x06,0x18,0x01,0x00],
       17:[0x55,0x55,0x05,0x06,0x19,0x01,0x00],
       18:[0x55,0x55,0x05,0x06,0x1A,0x01,0x00]}
final = 0
flag0 = 0
myBase = {1: [1, 1, 1],
          2: [1, 2, 1],
          3: [1, 3, 1],
          4: [2, 1, 1],
          5: [2, 2, 1],
          6: [2, 3, 1],
          7: [3, 1, 1],
          8: [3, 2, 1],
          9: [3, 3, 1]}

group = {0: [1, 2, 3],
         1: [4, 5, 6],
         2: [7, 8, 9],
         3: [1, 4, 7],
         4: [2, 5, 8],
         5: [3, 6, 9],
         6: [1, 5, 9],
         7: [3, 5, 7]}

result = [0, 0, 0, 0, 0, 0, 0, 0]


def led0on():
    base.leds[0].on()
    base.leds[1].off()
    base.leds[2].off()
    base.leds[3].off()

def led1on():
    base.leds[1].on()
    base.leds[0].off()
    base.leds[2].off()
    base.leds[3].off()
    
def led2on():
    base.leds[2].on()
    base.leds[1].off()
    base.leds[0].off()
    base.leds[3].off()
    
def led3on():
    base.leds[3].on()
    base.leds[1].off()
    base.leds[2].off()
    base.leds[0].off()


def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i
  
    return num


def my_sum(dic, list1):
    list1[0] = dic[1][2] + dic[2][2] + dic[3][2]
    list1[1] = dic[4][2] + dic[5][2] + dic[6][2]
    list1[2] = dic[7][2] + dic[8][2] + dic[9][2]
    list1[3] = dic[1][2] + dic[4][2] + dic[7][2]
    list1[4] = dic[2][2] + dic[5][2] + dic[8][2]
    list1[5] = dic[3][2] + dic[6][2] + dic[9][2]
    list1[6] = dic[1][2] + dic[5][2] + dic[9][2]
    list1[7] = dic[3][2] + dic[5][2] + dic[7][2]
    return list1


def check(list2):
    for i in range(8):
        if list2[i] == 0:
            print('Human wins.')
            return 1

        elif list2[i] == 6:
            print('Machine wins.')
            return 1


while 1:
    
    try:
        
    # Capture webcam image

        ret, frame_vga = videoIn.read()

    #Display webcam image via HDMI Out
        if (ret):      
            outframe = hdmi_out.newframe()
            outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        #hdmi_out.writeframe(outframe)
        else:
            raise RuntimeError("Failed to read from camera.")
        
    #Apply the gesture detection to the input.
        
        frame = frame_vga
        kernel = np.ones((3,3),np.uint8)
    
    #define region of interest
        roi=frame[100:400, 100:400]
        
        
        cv2.rectangle(frame,(100,100),(400,400),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
        
         
    # define range of skin color in HSV
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
        
    #extract skin colur imagw  
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
   
        
    #extrapolate the hand to fill dark spots within
        mask = cv2.dilate(mask,kernel,iterations = 4)
        
    #blur the image
        mask = cv2.GaussianBlur(mask,(5,5),100) 
        
        
        
    #find contours
        _,contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
       
        
    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
        #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
      
    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100
    
     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)
        
    # l = no. of defects
        l=0
        
    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)
            
            
            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            #distance between point and convex hull
            d=(2*ar)/a
            
            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
        
            # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
            if angle <= 90 and d>30:
                l += 1
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)
            
            
        l+=1
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                if arearatio<12:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('0')
                    flag+=1
                elif arearatio<17.5:
                    cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('6')
                    flag+=1
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('1')
                    flag+=1
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            cmds.extend('2')
            flag+=1
        elif l==3:
         
                if arearatio<27:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('3')
                    flag+=1
                else:
                    cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    cmds.extend('7')
                    flag+=1
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            cmds.extend('4')
            flag+=1
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            cmds.extend('5')
            flag+=1
        elif l==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        else :
            cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        
    # Output OpenCV results via HDMI
        outframe[0:480,0:640,:] = frame[0:480,0:640,:]
        hdmi_out.writeframe(outframe)

    except:
        pass

    if flag==60:
        actcmd=most_frequent(cmds)
        
        if(base.switches[1].read()==1):
            
            #print('state:',state,'    actcmd:',actcmd)
            
            if state==1:
                led0on()
                if actcmd=='1':
                    x='1'
                    state=2
                elif actcmd=='2':
                    x='2'
                    state=2
                elif actcmd=='3':
                    x='3'
                    state=2
                
            elif state==2:
                led1on()
                if actcmd=='5':
                    state=3
                    print('enter state 3')
                    
            elif state==3:
                led2on()
                if actcmd=='1':
                    y='1'
                elif actcmd=='2':
                    y='2'
                elif actcmd=='3':
                    y='3'
                
                if (flag0 == 0)and((actcmd == '1')or(actcmd == '2')or(actcmd == '3')):               
                    
                    if x == '1':
                        if y == '1':
                            myBase[1][2] = 0
                            print('act 1')
                            lib.uart_write(device,act[1], len(act[1]))
                            time.sleep(5)
            
                        elif y == '2':
                            myBase[2][2] = 0
                            print('act 2')
                            lib.uart_write(device,act[2], len(act[2]))
                            time.sleep(5)
            
                        elif y == '3':
                            myBase[3][2] = 0
                            print('act 3')
                            lib.uart_write(device,act[3], len(act[3]))
                            time.sleep(5)
            
                    elif x == '2':
                        if y == '1':
                            myBase[4][2] = 0
                            print('act 4')
                            lib.uart_write(device,act[4], len(act[4]))
                            time.sleep(5)
            
                        elif y == '2':
                            myBase[5][2] = 0
                            print('act 5')
                            lib.uart_write(device,act[5], len(act[5]))
                            time.sleep(5)
            
                        elif y == '3':
                            myBase[6][2] = 0
                            print('act 6')
                            lib.uart_write(device,act[6], len(act[6]))
                            time.sleep(5)
            
                    elif x == '3':
                        if y == '1':
                            myBase[7][2] = 0
                            print('act 7')
                            lib.uart_write(device,act[7], len(act[7]))
                            time.sleep(5)
            
                        elif y == '2':
                            myBase[8][2] = 0
                            print('act 8')
                            lib.uart_write(device,act[8], len(act[8]))
                            time.sleep(5)
            
                        elif y == '3':
                            myBase[9][2] = 0
                            print('act 9')
                            lib.uart_write(device,act[9], len(act[9]))
                            time.sleep(5)
            
                    my_sum(myBase, result)
                    final = check(result)
                    flag0 = 1
                
                if flag0 == 1:
                    p = 0
                    print(result)
            
                    if p == 0:
                        for j in range(8):
                            if result[j] == 5:
                                print('result 5')
                                for k in range(3):
                                    ta = group[j][k]
                                    if myBase[ta][2] == 1:
                                        myBase[ta][2] = 2
                                        actNum = 9 + group[j][k]
                                        print('act', actNum)
                                        lib.uart_write(device,act[actNum], len(act[actNum]))
                                        p = p + 1
                                break
            
                    if p == 0:
                        for j in range(8):
                            if (result[j] == 1)and(result[j] != 5):
                                print('result 1')
                                for k in range(3):
                                    ta = group[j][k]
                                    if myBase[ta][2] == 1:
                                        myBase[ta][2] = 2
                                        actNum = 9 + group[j][k]
                                        print('act', actNum)
                                        lib.uart_write(device,act[actNum], len(act[actNum]))
                                        p = p + 1
                                break
            
                    if p == 0:
                        for j in range(8):
                            if (result[j] == 2)and(result[j] != 5)and(result[j] != 1):
                                print('result 2')
                                print(myBase[5][2])
                                if myBase[5][2] != 1:
                                    print('1')
                                    if (myBase[1][2] != 1)and(myBase[3][2] != 1)and(myBase[7][2] != 1)and(myBase[9][2] != 1):
                                        if myBase[2][2] == 1:
                                            myBase[2][2] = 2
                                            print('act 11')
                                            lib.uart_write(device,act[11], len(act[11]))
                                            p = p + 1
                                            break
            
                                        elif (myBase[6][2] == 1)and(myBase[2][2] != 1):
                                            myBase[6][2] = 2
                                            print('act 15')
                                            lib.uart_write(device,act[actNum15], len(act[15]))
                                            p = p + 1
                                            break
            
                                        elif (myBase[4][2] == 1)and(myBase[6][2] != 1):
                                            myBase[4][2] = 2
                                            print('act 13')
                                            lib.uart_write(device,act[13], len(act[13]))
                                            p = p + 1
                                            break
            
                                        elif (myBase[8][2] == 1)and(myBase[4][2] != 1):
                                            myBase[8][2] = 2
                                            print('act 17')
                                            lib.uart_write(device,act[17], len(act[17]))
                                            p = p + 1
                                            break
                                    else:
                                        print('2')
                                        if myBase[1][2] == 1:
                                            myBase[1][2] = 2
                                            print('act 10')
                                            lib.uart_write(device,act[10], len(act[10]))
                                            p = p + 1
                                            break
            
                                        elif (myBase[3][2] == 1)and(myBase[1][2] == 1):
                                            myBase[3][2] = 2
                                            print('act 12')
                                            lib.uart_write(device,act[12], len(act[12]))
                                            p = p + 1
                                            break
            
                                        elif (myBase[7][2] == 1)and(myBase[3][2] == 1):
                                            myBase[7][2] = 2
                                            print('act 16')
                                            lib.uart_write(device,act[16], len(act[16]))
                                            p = p + 1
                                            break
            
                                        elif (myBase[9][2] == 1)and(myBase[7][2] == 1):
                                            myBase[9][2] = 2
                                            print('act 18')
                                            lib.uart_write(device,act[18], len(act[18]))
                                            p = p + 1
                                            break
                                else:
                                    myBase[5][2] = 2
                                    print('act 14')
                                    lib.uart_write(device,act[14], len(act[14]))
                                    p = p + 1
                                    break
            
                    if p == 0:
                        for j in range(8):
                            if (result[j] == 4)and(result[j] != 2)and(result[j] != 5)and(result[j] != 1):
                                print('result 4')
                                ta = group[j][0]
                                tb = group[j][1]
                                tc = group[j][2]
                                if (myBase[ta][2] != 0) and (myBase[tb][2] != 0) and (myBase[tc][2] != 0):
            
                                    for k in range(3):
                                        ta = group[j][k]
                                        if myBase[ta][2] == 1:
                                            myBase[ta][2] = 2
                                            actNum = 9 + group[j][k]
                                            print('act', actNum)
                                            lib.uart_write(device,act[actNum], len(act[actNum]))
                                            p = p + 1
                                            break
                                    break
            
                    if p == 0:
                        t = 0
                        if t == 0:
                            for g in range(1, 10):
                                if myBase[g][2] == 1:
                                    myBase[g][2] = 2
                                    actNum = 9 + g
                                    print('act', actNum)
                                    lib.uart_write(device,act[actNum], len(act[actNum]))
                                    p = p + 1
                                    t = t + 1
                                    break
                        if t == 0:
                            temp_sum = 0
                            for j in range(8):
                                temp_sum = temp_sum + result[j]
                            if temp_sum % 2 == 0:
                                print('No one wins!')
                                p = p + 1
                                t = t + 1
                                break
            
                    my_sum(myBase, result)
                    final = check(result)
                    flag0 = 0
                    state = 4
                
                print('Game algorithm cycle')
                
                if final == 1:
                    break
                
            elif state==4:
                led3on()
                if actcmd=='7':
                    state=1
                    print('enter state 1')
                
        cmds.clear()
        flag=0
    
    if(base.buttons[0].read()==1):
        break

enter state 1
enter state 3
Game algorithm cycle
act 1
[2, 3, 3, 2, 3, 3, 2, 3]
result 2
1
act 14
Game algorithm cycle


In [ ]:
videoIn.release()
hdmi_out.stop()
del hdmi_out